# Section 2.9 – Travaux pratiques

Ce notebook présente des scripts R correspondant aux exercices du chapitre 2.9 du polycopié. 
Les commentaires commençant par `#` expliquent pas à pas ce qui est fait, de manière à pouvoir suivre sans revenir constamment au document.


## 2.9.1 Biais et variance

### Exercice 11 – Calcul de la variance empirique
La variance empirique d’une série d’observations $(x_1, \ldots, x_n)$ est donnée par
$$s_n'^2 = \frac{1}{n} \sum_{i=1}^n (x_i - \bar{x}_n)^2$$
ce qui diffère légèrement de la variance corrigée utilisée par R.


In [ ]:

# Exemple d'un petit jeu de données
x <- c(4.2, 5.0, 3.8, 4.7, 5.3)

# Variance corrigée de R (diviseur n-1)
var_R <- var(x)
var_R

# Fonction qui calcule la variance empirique (diviseur n)
variance_empirique <- function(x) {
  n <- length(x)
  m <- mean(x)
  return(sum((x - m)^2) / n)
}

var_emp <- variance_empirique(x)
var_emp


On peut comparer ces deux résultats : `var()` de R divise par $n - 1$ alors que notre fonction `variance_empirique()` divise par $n$. La seconde correspond à la vraie **variance empirique**.


### Exercice 13 – Moyenne d’un échantillon
La moyenne d’un échantillon $\bar{X}_n$ est un estimateur sans biais de la moyenne théorique.  
Dans le cas d’une loi exponentielle $\mathcal{E}(\lambda)$, on peut le vérifier par simulation.


In [ ]:

# Simulation pour illustrer la convergence de la moyenne d'un échantillon
set.seed(42)           # pour reproduire les résultats
n <- 1000              # taille de l'échantillon
lambda <- 1/2          # paramètre de la loi exponentielle (moyenne = 2)

# Génération de n observations de loi exponentielle
X <- rexp(n, rate = lambda)

# Calcul de la moyenne empirique
mean_emp <- mean(X)
mean_emp

# Valeur théorique de la moyenne : 1/lambda
mean_theo <- 1 / lambda
mean_theo


On observe que la moyenne empirique est proche de la moyenne théorique lorsque $n$ est grand.


### Exercice 14 – Estimateur sans biais de la variance
R utilise par défaut la variance corrigée (diviseur \(n-1\)) comme estimateur sans biais de la variance de la population. La fonction `var()` renvoie donc déjà un estimateur sans biais.  

Pour illustrer cela, on peut calculer la variance empirique et la variance corrigée sur un échantillon.


In [ ]:

# Génération d'un échantillon normal
set.seed(123)
n <- 20
X <- rnorm(n, mean = 0, sd = 1)

# Variance empirique (diviseur n)
var_emp <- variance_empirique(X)

# Variance corrigée (diviseur n-1)
var_corrigee <- var(X)

c(var_emp = var_emp, var_corrigee = var_corrigee)


La variance corrigée est systématiquement un peu plus grande que la variance empirique, ce qui corrige le biais.


## 2.9.2 Comparaison des estimateurs du paramètre de position

Nous comparons trois estimateurs de la position $\theta$ : la moyenne, la médiane et le « mid‑range » (moyenne des minimum et maximum).  

Le code suivant génère un échantillon de taille 1000 issu d’une loi normale $\mathcal{N}(5,4)$ et trace l’histogramme de l’échantillon ainsi que la densité théorique.


In [ ]:

# Paramètres
n <- 1000
mu <- 5
sigma <- 2

# Génération de l'échantillon
X <- rnorm(n, mean = mu, sd = sigma)

# Histogramme de l'échantillon
hist(X, breaks = 20, freq = FALSE, col = "cyan",
     main = "Distribution de l'échantillon", xlab = "x")

# Densité normale théorique superposée
curve(dnorm(x, mean = mu, sd = sigma), add = TRUE, col = "red")


La densité théorique (courbe rouge) recouvre bien l'histogramme, ce qui confirme que la simulation est conforme à la loi normale $\mathcal{N}(5,4)$. Dans ce cas, la valeur de $\theta$ est simplement la moyenne $\mu = 5$.


Nous définissons maintenant une fonction `location_estimator()` qui, pour un vecteur de données `U` et une valeur théorique `theta`, calcule la moyenne et la médiane cumulées au fur et à mesure que la taille d'échantillon augmente. Des graphiques permettent de visualiser la convergence.


In [ ]:

location_estimator <- function(U, theta) {
  n <- length(U)
  # Moyenne cumulative
  theta1 <- cumsum(U) / (1:n)
  
  # Médiane cumulative
  theta2 <- numeric(n)
  for (i in 1:n) {
    theta2[i] <- median(U[1:i])
  }
  
  # Tracé des moyennes cumulées
  plot(1:n, theta1, type = "l", main = "Moyenne cumulée", ylab = "estimateur", xlab = "i")
  abline(h = theta, col = "darkred", lty = 2)
  
  # Tracé des médianes cumulées
  plot(1:n, theta2, type = "l", main = "Médiane cumulée", ylab = "estimateur", xlab = "i")
  abline(h = theta, col = "darkred", lty = 2)
  
  # Retourne les deux séries sous forme de matrice
  return(cbind(theta1, theta2))
}

# Utilisation de la fonction
set.seed(1)
U <- rnorm(n, mean = mu, sd = sigma)
est <- location_estimator(U, theta = mu)

# Afficher les premières lignes du résultat
head(est)


Le vecteur `theta1` correspond aux moyennes cumulées et `theta2` aux médianes cumulées. L’option `type="l"` dans la fonction `plot()` demande de tracer une ligne continue plutôt que des points.

La matrice retournée contient en colonne 1 la moyenne cumulée et en colonne 2 la médiane cumulée pour chaque valeur de `i`.


Pour comparer les trois estimateurs (moyenne, médiane et mid‑range) sur plusieurs échantillons, la fonction ci‑dessous génère `N` échantillons de taille `n` et trace des diagrammes en boîte des estimateurs.


In [ ]:

compare_estimators <- function(N, n, dist = "norm") {
  # Génération de N échantillons de taille n
  if (dist == "norm") {
    X <- matrix(rnorm(N * n, mean = 5, sd = 2), nrow = N)
  } else if (dist == "unif") {
    X <- matrix(runif(N * n, min = 0, max = 10), nrow = N)
  } else if (dist == "cauchy") {
    X <- matrix(rcauchy(N * n, location = 5, scale = 2), nrow = N)
  } else {
    stop("Distribution non supportée")
  }
  
  # Calcul des estimateurs pour chaque échantillon
  theta1 <- apply(X, 1, mean)
  theta2 <- apply(X, 1, median)
  theta3 <- (apply(X, 1, min) + apply(X, 1, max)) / 2
  
  # Diagrammes en boîte
  boxplot(theta1, theta2, theta3, names = c("moyenne", "médiane", "mid-range"),
          col = "cyan", main = paste("Distribution des estimateurs (", dist, ")", sep = ""))
  
  return(list(theta1 = theta1, theta2 = theta2, theta3 = theta3))
}

# Exemple d'appel : comparaison sur 200 échantillons
set.seed(2)
res <- compare_estimators(N = 200, n = 1000, dist = "norm")


On peut modifier l’argument `dist` pour tester d’autres distributions (uniforme ou Cauchy). Le résultat de la fonction `apply()` est utilisé pour calculer un statistic pour chaque ligne de la matrice : ici la moyenne, la médiane ou les valeurs minimale et maximale.


## 2.9.3 Intervalles de confiance

Pour vérifier par simulation que
\[\theta_n = \frac{\bar{X}_n - \mu}{S_n / \sqrt{n}}\]
suit bien une loi de Student à \((n-1)\) degrés de liberté lorsque les données proviennent d'une loi normale, on définit la fonction suivante :


In [ ]:

check_student <- function(N, n, mu, sigma) {
  # Génère une matrice X de dimension N x n
  X <- matrix(rnorm(N * n, mean = mu, sd = sigma), nrow = N)
  
  # Calcule la statistique de test pour chaque échantillon
  t_vals <- sqrt(n) * (apply(X, 1, mean) - mu) / apply(X, 1, sd)
  
  return(t_vals)
}

# Simulation avec N=5000 échantillons de taille 30
set.seed(3)
N <- 5000
n <- 30
mu <- 2
sigma <- 3

t_values <- check_student(N, n, mu, sigma)

# Histogramme des valeurs simulées et densité théorique
hist(t_values, breaks = 30, freq = FALSE, col = "cyan",
     main = "Statistique t simulée", xlab = "t")
curve(dt(x, df = n - 1), add = TRUE, col = "red", lwd = 2)

# Quantiles empirique à 2,5% et 97,5%
quantile(t_values, c(0.025, 0.975))


Les quantiles simulés se rapprochent de ceux de la loi de Student lorsque `N` est grand. 

On peut alors utiliser les quantiles théoriques pour calculer un intervalle de confiance. Par exemple, pour la moyenne d’un échantillon gaussien:
\[\left[\bar{X}_n - S_n \frac{b}{\sqrt{n}},\; \bar{X}_n - S_n \frac{a}{\sqrt{n}}\right]\]
donne un intervalle de confiance à 95 %. Le code ci‑dessous illustre ce calcul sur les données de résistance du ciment.


In [ ]:

# Données de résistance à la compression (en kg/cm^2)
X_ciment <- c(19.6, 19.9, 20.4, 19.8, 20.5, 21.0, 18.5, 19.7, 18.4, 19.4)

n <- length(X_ciment)
barX <- mean(X_ciment)
S <- sd(X_ciment)

# Quantiles théoriques de la loi de Student pour un IC à 95 %
alpha <- 0.05
a <- qt(alpha/2, df = n - 1)       # quantile 2.5%
b <- qt(1 - alpha/2, df = n - 1)   # quantile 97.5%

IC <- c(barX - S * b / sqrt(n), barX - S * a / sqrt(n))
IC


Ce code fournit l’intervalle de confiance pour la moyenne vraie \(\mu\) avec un niveau de confiance de 95\u00A0%. On peut adapter les valeurs de \(\alpha\), `n` et des paramètres pour d’autres situations.


## 2.9.4 Calcul de l’EMV pour une loi exponentielle

À partir de l’échantillon de durées de vie (en années) suivant :


In [ ]:

# Échantillon de durées de vie d'un lave‑linge
X <- c(10.273, 1.035, 63.952, 33.153, 21.708, 1.612, 22.216, 17.682, 4.730, 2.236,
       0.746, 22.762, 23.455, 30.993, 30.138, 3.140, 12.884, 40.893, 35.560, 5.999,
       5.962, 7.069, 23.743, 30.881, 6.647, 2.393, 3.736, 0.866, 46.587, 3.728,
       4.173, 10.127, 40.502, 15.275, 0.030, 2.517, 3.134, 15.530, 15.156, 7.919,
       32.617, 18.881, 1.221, 0.360, 8.641, 36.070, 16.217, 10.520, 21.354, 1.366)

# 1. Représenter graphiquement la distribution
hist(X, breaks = 20, freq = FALSE, col = "lightblue",
     main = "Durée de vie d'un lave‑linge", xlab = "Années")

# 2. Estimation par maximum de vraisemblance
# On définit la log-vraisemblance négative de la loi exponentielle
log.vrais.neg <- function(theta = 1) {
  if (theta > 0) {
    return(-sum(dexp(X, rate = theta, log = TRUE)))
  } else {
    return(NA)
  }
}

# Charger le package stats4 (si nécessaire)
# install.packages("stats4")  # Décommenter si le package n'est pas installé
library(stats4)

# Estimation de l'EMV avec la fonction mle()
fit <- mle(log.vrais.neg)
summary(fit)

# 3. Comparaison avec l'estimateur simple 1/ar{X}
theta_hat_moments <- 1 / mean(X)
theta_hat_moments

# Superposer la densité théorique obtenue à partir de l'estimation
curve(dexp(x, rate = coef(fit)), add = TRUE, col = "red", lwd = 2)


Dans ce script :

- La fonction `log.vrais.neg()` calcule l’opposé du log‑de la vraisemblance pour la loi exponentielle.  
- La fonction `mle()` (package `stats4`) trouve la valeur de $\theta$ qui minimise cette quantité, c’est‑à‑dire l’EMV.  
- On compare ensuite cette estimation à l’estimateur $\hat{\theta} = 1/\bar{X}$ issu de la méthode des moments.  
- Enfin, la densité théorique de la loi exponentielle ajustée est superposée à l’histogramme des données.

**Remarque :** l’estimateur de maximum de vraisemblance n’est pas toujours unique et peut être biaisé. Il possède toutefois de bonnes propriétés asymptotiques lorsque les conditions de régularité sont satisfaites.
